### Прочитаем train

In [110]:
train = pd.read_csv('train_merged_full_morphed.csv')

In [111]:
train.drop(columns=['fd_slug_id1','fd_slug_id2',\
                    'fd_slug1','fd_slug2','Unnamed: 0' ],axis=1, inplace=True) #07.01.19

train['price'] = train['price']/100

train['morph_description'] = train['morph_description'].fillna('').astype('object')

In [112]:
train.head().T

,0,1,2,3,4
id,3edeb34cf93f490ff760af85,c98febd50dad3cc0ffc86085,ade01e13912a46a99134cc75,ab3e6941c11304c1519aef75,252452a91c944a22c276d995
category,9,22,22,22,22
subcategory,914,2202,2204,2209,2204
name,Сумка DG,Комплект,Пальтишко демисезонное,Attipas,Жилет теплый
description,NaN,8-12 лет,"На девочку 1,5 г,состояние хорошее","Размер-135mm, euro-22.5.Прочная, мягкая, не ск...",NaN
price,1999,350,300,800,500
fd_name1,Женский гардероб,Детский гардероб,Детский гардероб,Детский гардероб,Детский гардероб
fd_name2,Аксессуары,Аксессуары,Верхняя одежда,Обувь,Верхняя одежда
morph_description,,год,девочка год состояние хороший,размер прочный мягкий скользящий обувь маленьк...,
morph_name,сумка dg,комплект,пальтишко демисезонный,attipas,жилет тёплый


In [70]:
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

In [71]:
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion

 Зададим селектор

In [72]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X_, y=None): # 
        return self
    def transform(self, X_):
        return X_.loc[:, self.attribute_names].values # 

In [73]:
class DenseTransformer(BaseEstimator,TransformerMixin):

    def transform(self, X, y=None, **fit_params):
        #return X.todense()
        return X.toarray()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

Зададим списки столбцов

In [109]:
# получим список столбцов 
full_columns_list = list(train) 

# список числовых категориальных столбцов
num_cat_columns = full_columns_list[1:3] # ['category', 'subcategory']

# целвое значение
target = ['price']
print('num_cat_columns :',num_cat_columns)
print('tfidf_columns   :',tfidf_columns)

other = list(set(full_columns_list) - set(num_cat_columns)  - set(tfidf_columns) - set(target))
print('Остаток         :', other)

num_cat_columns : ['category', 'subcategory']
tfidf_columns   : ['morph_description', 'morph_name']
Остаток         : ['name', 'fd_name2', 'fd_name1', 'id', 'description', 'morph_name_2']


<font color="magenta" size="5">CatBoost.</font><br> 


In [76]:
from catboost import CatBoostClassifier
catbost_model = CatBoostClassifier(depth = 5, num_trees=150, learning_rate = 0.11,verbose=False)

In [87]:
model = Pipeline([  
('selector', ColumnSelector(num_cat_columns)), # определим категориальные числовые столбцы: ['category', 'subcategory']
('OHE', OneHotEncoder()),
('denser',DenseTransformer()),
('catbost', catbost_model)
])

In [81]:
y = train['price']
X_train = train.iloc[:1000000,:]
y_train = y.iloc[:1000000]

X_test = train.iloc[1000000:,:]
y_test = y.iloc[1000000:]

In [83]:
print(X_train.shape)
print(y_train.shape)

(1000000, 11)
(1000000,)


In [88]:
%%time 
model.fit(X_train, np.log(y_train+1))

/home/vladimirpolanski6/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


CPU times: user 6min 57s, sys: 12.5 s, total: 7min 9s
Wall time: 1min 30s


Pipeline(memory=None,
     steps=[('selector', ColumnSelector(attribute_names=['category', 'subcategory'])), ('OHE', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values='auto', sparse=True)), ('denser', DenseTransformer()), ('catbost', <catboost.core.CatBoostClassifier object at 0x7f0e58b6a5c0>)])

In [89]:
y_hat_log = model.predict(X_test)
y_hat_log.shape

(748890,)

In [90]:
y_hat = np.exp(y_hat_log)-1

In [91]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error 

In [93]:
print(mean_absolute_error(y_test,y_hat))
#print(mean_squared_log_error(y_test,y_hat))

317705.64384058473


In [100]:
y_test_df = pd.DataFrame(y_test)
y_hat_df = pd.DataFrame(y_hat, index=range(1000000,len(y_hat)+1000000))

In [105]:
print(y_hat_df.shape)
print(y_test_df.shape)

(748890, 1)
(748890, 1)


In [106]:
y_test_df.head()

,price
1000000,3000.0
1000001,120.0
1000002,500.0
1000003,12000.0
1000004,0.0


In [108]:
y_hat_df.head(50)

,0
1000000,1.718282
1000001,1.718282
1000002,1.718282
1000003,1.718282
1000004,1.718282
1000005,1.718282
1000006,1.718282
1000007,1.718282
1000008,1.718282
1000009,1.718282


In [ ]:
# объединим предсказанную часть трейна, цену и предсказанную цену
train_predicted = pd.concat([train__, y_hat_df], axis=1)

In [ ]:
train_predicted.head(50)

In [ ]:
train_predicted.shape